In [ ]:
# Set your OpenAI API key (required for openai/gpt-4o-mini).
# Option 1: Set in this notebook (replace with your key, avoid committing it):
import os
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY", "sk-your-key-here")

# Option 2: Set in your shell before starting Jupyter, e.g.:
#   export OPENAI_API_KEY="sk-your-key-here"
# Then you can skip the line above or leave it as-is (getenv will use the exported value).

In [11]:
import dspy

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

In [12]:

qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

print(response.response)

2026/02/10 13:17:52 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


RuntimeError: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [2]:
import orjson
import requests
from pathlib import Path

# Save in this repo so the path is predictable (run this cell from workspace/dspy or set path below)
DATA_PATH = Path("ragqa_arena_tech_examples.jsonl").resolve()
print(f"Data file: {DATA_PATH}")

Data file: /Users/eddiej@nisos.com/mespace/dspy/ragqa_arena_tech_examples.jsonl


In [ ]:
URL = "https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl"
if not DATA_PATH.exists():
    print(f"Downloading to {DATA_PATH}...")
    r = requests.get(URL, stream=True)
    r.raise_for_status()
    with open(DATA_PATH, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
else:
    print(f"Using existing file: {DATA_PATH}")

In [3]:
with open(DATA_PATH) as f:
    data = [orjson.loads(line) for line in f]
    
data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

In [4]:
import dspy
data = [dspy.Example(**d).with_inputs('question') for d in data]

In [5]:
data

[Example({'question': 'why igp is used in mpls?', 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.", 'gold_doc_ids': [2822, 2823]}) (input_keys={'question'}),
 Example({'question': 'how do you achieve a numeric versioning scheme with git?', 'response': "Take a look at the process of using 'git-describe' by way of GIT-VERSION-GEN and how you can add this via your build process when you tag your release. \nFor guidance, can also refer to the provided blog at http://cd34.com/blog/programming/using-git-to-generate-an-automatic-version-number/.", 'gold_doc_ids': [3965]}) (input_keys={'question'}),
 Example({'question': 'why are my text messages coming up as maybe

In [7]:
import random

random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

len(trainset), len(devset), len(testset)



(200, 300, 500)

In [9]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

NameError: name 'cot' is not defined